# Import Libraries

In [1]:

import os
import pandas as pd
import torch
import numpy as np
from Preprocess import Process
from NeuralNetworkDIY import ClassifierNetwork, Model
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
from PIL import Image
torch.cuda.empty_cache()

In [2]:
print(torch.cuda.is_available())

True


# Parameters for the dataset

In [3]:
RAW_DIR = "data/raw/140k faces"
PROCESSED_DIR = "data/processed"
IMAGE_SIZE=(128,128)

# Image Preprocessing

In [4]:
# process=Process(RAW_DIR, PROCESSED_DIR, IMAGE_SIZE, delete_process_dir=True, batch_size=1000, aug_amount=3, filters=False)
# process.run(0.95, 0.05)

# DataLoader and Custom Dataset

In [5]:
class CustomImageDataset(Dataset):
    def __init__(self, directory):
        self.directory = directory
        self.image_paths = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith((".png", ".jpg", ".jpeg"))]

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        img = Image.open(img_path).convert("RGB")

        transform = transforms.ToTensor()
        
        img= transform(img)
        # Extract label from the filename (fake -> 1, real -> 0)
        label = 1 if "fake" in os.path.basename(img_path).lower() else 0

        return img, label, img_path

In [6]:
train_data = CustomImageDataset(directory=f'{PROCESSED_DIR}/{IMAGE_SIZE[0]}/train/rgb')

train_loader = DataLoader(train_data, batch_size=1000, shuffle=True)

# Creating the network, model, training, predicting

In [7]:
network=ClassifierNetwork(IMAGE_SIZE)

In [8]:
model=Model(network, 10)

In [9]:
model.train(train_loader)

Epoch 1/10, Loss: 0.7234056819767374
Epoch 2/10, Loss: 0.44661855315725646
Epoch 3/10, Loss: 0.33239736798426106
Epoch 4/10, Loss: 0.2694991558483318
Epoch 5/10, Loss: 0.22565780639716634
Epoch 6/10, Loss: 0.19614586138616164
Epoch 7/10, Loss: 0.17321705922177508
Epoch 8/10, Loss: 0.15442254811150258
Epoch 9/10, Loss: 0.13892722494678583
Epoch 10/10, Loss: 0.12499064688722111


In [10]:
torch.save(model, 'ADL_model.pth')